# Example 07: Upsampling calculations

In this example, upsampling calculations which can be used to switch a system between two different interatomic potentials is illustrated. The input file is shown below:

```
element: ['Cu']
mass: [63.546]
calculations:
- mode: alchemy 
  temperature: [600]
  pressure: [0]
  lattice: [FCC]
  repeat: [5, 5, 5]
  state: [solid]
  nsims: 1
  
md:
  pair_style: [eam/fs, eam/alloy]
  pair_coeff: ["* * ../potentials/Cu1.eam.fs Cu", "* * ../potentials/Cu01.eam.alloy Cu"]
  timestep: 0.001
  tdamp: 0.1
  pdamp: 0.1
  te: 10000
  ts: 15000

queue:
  scheduler: local
  cores: 4
  commands:
    - conda activate calphy
```

The major change is that both `pair_style` and `pair_coeff` keywords have two arguments. These are the two potentials between which the transformation will be carried out.

The first potential is a Finnis-Sinclair (FS) potential for copper:  

[M.I. Mendelev, M.J. Kramer, C.A. Becker, and M. Asta (2008), "Analysis of semi-empirical interatomic potentials appropriate for simulation of crystalline and liquid Al and Cu", Philosophical Magazine, 88(12), 1723-1750.](https://doi.org/10.1080/14786430802206482)

The second potential is an EAM:  

[Mishin, Y., M. J. Mehl, D. A. Papaconstantopoulos, A. F. Voter, and J. D. Kress. “Structural Stability and Lattice Defects in Copper: Ab Initio , Tight-Binding, and Embedded-Atom Calculations.” Physical Review B 63, no. 22 (May 21, 2001): 224106.](https://doi.org/10.1103/PhysRevB.63.224106)  

If we know the free energy of the FS potential at a given temperature, we can calculate the free energy of the EAM through upsampling calculations. Upsampling calculations generally need only much less switching time, thus is quite useful in the case of expensive interatomic potentials, such as machine learning potentials.

We start by calculating the free energy of the FS potential. The input file is available at `pot1/input-fe.yaml`. As usual, the calculation can be run using `calphy -i input-fe.yaml`. We can now load the `report.yaml` file and check the free energy.

In [1]:
import yaml
import numpy as np

In [2]:
with open('pot1/fe-FCC-600-0/report.yaml', 'r') as fin:
    pot1 = yaml.safe_load(fin)

In [3]:
pot1['results']['free_energy']

-3.43894689000222

Now we can transform FS to EAM potential (see input file above). After running the calculation, we can check the free energy of this alchemical transformation.

In [4]:
with open('alchemy-FCC-600-0/report.yaml', 'r') as fin:
    alchemy = yaml.safe_load(fin)

In [5]:
alchemy['results']['free_energy']

-0.2544935936478223

The free energy of the EAM potential, $F_\mathrm{EAM} = F_\mathrm{FS} + F_\mathrm{upsampling}$

In [6]:
pot1['results']['free_energy']+alchemy['results']['free_energy']

-3.6934404836500425

We can verify this calculation by directly computing the free energy of the EAM potential. The input file is available at `pot2/input-fe.yaml`. 

In [7]:
with open('pot2/fe-FCC-600-0/report.yaml', 'r') as fin:
    pot2 = yaml.safe_load(fin)

In [8]:
pot2['results']['free_energy']

-3.6942301262722053

We can see that the directly calculation is in meV agreement with the upsampling calculations.